In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os

In [ ]:
def MSD_NGP_func(frame_prev, frame_cur):

    dr2 = []
    
#     part_counter = 0
    
    for particle in frame_cur['particle']:
        
#         if part_counter > 50:
#             break
        
        if (particle in frame_prev['particle'].unique()):
            mask1 = frame_cur['particle'] == particle
            x1 = frame_cur[mask1]['x'].item()
            y1 = frame_cur[mask1]['y'].item()

            mask2 = frame_prev['particle'] == particle
            x2 = frame_prev[mask2]['x'].item()
            y2 = frame_prev[mask2]['y'].item()

            dr2.append( (x2 - x1)**2 + (y2 - y1)**2 )
            
#         part_counter += 1
    
    
    return dr2
    

In [ ]:
def Assembly(lag_frame):

    global data
    global max_frame
    
    dr2 = []
    
    for frame in range(1999+lag_frame, max_frame): #range(lag_frame, max_frame):
        frame_prev = data[data['frame'] == frame - lag_frame]
        frame_cur  = data[data['frame'] == frame]
        
        dr2_cur = MSD_NGP_func(frame_prev, frame_cur)
        dr2.extend(dr2_cur)
        
#     print(f'max(dr2) = {max(dr2)}')
            
    if len(dr2) > 0:
        
        dr2_mean = np.mean(dr2)
        msd = dr2_mean
        
#         dr2_lim = [x for x in dr2 if x < 10.]
        dr4 = [x**2 for x in dr2]# if x < 1.]
#         dr2_lim_mean = np.mean(dr2_lim)
        dr4_mean = np.mean(dr4)
        
        ngp = (3 * dr4_mean/(5 * (dr2_mean**2))) - 1
        
#         print(f'max(dr4) = {max(dr4)}')
        
#         print(f'd2_mean = {np.mean(dr2)}')
#         print(f'd4_mean = {np.mean(dr4)}')
#         print(ngp)
    else:
        msd = math.nan
        ngp = math.nan
    
#     print("\n")
    
    return msd, ngp

In [ ]:
# def Assembly(frameN):

#     global data
#     global max_frame
    
#     frame_prev = data[data['frame'] == 0]
    
#     frame_cur  = data[data['frame'] == frameN]
        
#     dr2 = MSD_NGP_func(frame_prev, frame_cur)
        
        
#     if len(dr2) > 0:
#         dr2_mean = np.mean(dr2)
#         msd = dr2_mean
        
#         dr4 = [x**2 for x in dr2]
#         dr4_mean = np.mean(dr4)
        
#         ngp = 3*dr4_mean/(5*(dr2_mean**2)) - 1
#     else:
#         msd = math.nan
#         ngp = math.nan

    
#     return msd, ngp

In [ ]:
def read_data():
    
    file_path = 'data_out/backup/feb20_S9_t_corrected.csv'
    
    usecols = ['y', 'x', 'frame', 'particle']
    
    data = pd.read_csv(file_path, usecols=usecols)
    
    
    return data

In [ ]:
%%time

# check if file already exists
# path = 'data_out/NGP_S5.dat'
# if os.path.isfile(path):
#     lag_frame, NGP = read_from_file(path)
# else:


data = read_data()
data_frame0 = data[data['frame'] == 0]
max_frame = data['frame'].values[-1] + 1

MSD = []
NGP = []
lag_frame = []

for lg in range(1, max_frame):
# lg = 1
# while lg < max_frame:
    
    assembly_MSD, assembly_NGP = Assembly(lg)

    if math.isnan(assembly_MSD) or math.isnan(assembly_NGP):
        print('no common particles')
        continue
    else:
        MSD.append(assembly_MSD)
        NGP.append(assembly_NGP)
        lag_frame.append(lg)
        print(f'lg = {lg} V')
        
#     if lg <= 10:
#         step = 1
#     elif lg <= 100:
#         step = 10
#     else:
#         step = 100
        
#     lg += step

In [ ]:
# convert to microns
px_to_micron = 0.06905
frame_to_sec = 1./120.

In [ ]:
lag_frame_sec = [i*frame_to_sec for i in lag_frame]
MSD_micron = [i*px_to_micron*px_to_micron for i in MSD]

In [ ]:
# # read from file MSD
# lag_fr_MSD_prev = []
# MSD_prev = []

# with open('data_out/backup/MSD_S5.dat') as f:
#     next(f)
#     for rows in f:
#         lag_fr_MSD_prev.append( float(rows.split(',')[0]) )
#         MSD_prev.append( float(rows.split(',')[1]) )

In [ ]:
MSD_trackpy = pd.read_csv('data_out/backup/feb20_S9_EMSD.csv')
# MSD_trackpy.head

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(MSD_trackpy['lag time t'], MSD_trackpy['dr^2'], marker='^', ms=8, label='trackpy')
# ax.plot(lag_fr_MSD_prev, MSD_prev, marker='o', ms=5, ls='--', label='MSD, S5, previous')
ax.plot(lag_frame_sec, MSD_micron, marker='*', ms=5, ls='--', label='MSD, S9')

ax.set(ylabel=r'$\alpha_2 (\tau)$',
      xlabel=r'$\tau$ [sec]')
ax.set_xscale('log')
ax.set_yscale('log')
plt.legend (fontsize=14)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(lag_frame_sec, NGP, marker='*', ms=5, ls='--', label='NGP, S9')

ax.set(ylabel=r'$\alpha_2 (\tau)$',
      xlabel=r'$\tau$ [sec]')
ax.set_xscale('log')
# ax.set_yscale('log')
plt.legend (fontsize=14)

In [ ]:
# write to file
DIR = 'data_out/'

file_MSD = DIR + 'MSD_S9.dat'
file_NGP = DIR + 'NGP_S9.dat'

data_out_dict_MSD = {"lag_frame[sec]": lag_frame_sec, "MSD_[px^2]": MSD}
data_out_dict_NGP = {"lag_frame[sec]": lag_frame_sec, "NGP_[dimensionless]": NGP}

data_out_MSD = pd.DataFrame(data_out_dict_MSD)
data_out_NGP = pd.DataFrame(data_out_dict_NGP)

data_out_MSD.to_csv(file_MSD, index=False)
data_out_NGP.to_csv(file_NGP, index=False)